In [1]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
import time

In [2]:
spark = SparkSession.builder.master('local[*]').appName('Opensea_transformation').getOrCreate()
json_paths='/home/roger/SB/Capstone/NFT_ETH_pipeline/Data/Raw/'
json_lst=[json_paths+'Cryptopunks',
    json_paths+'Bored_apes',
    json_paths+'Cool_cats',
    json_paths+'Meebits',
    json_paths+'Doodles',
]

In [7]:
df = spark.read.json('/home/roger/SB/Capstone/Open_capstone_git/data/CryptoPunks')         
df.show(1)  

+----------------------+-------------+--------------------+----------------+--------------------+--------------------+--------+-----------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+------------+----------------+---------+--------------------+--------------------+-----------+--------+--------------+-------+--------------------+------------+--------------------------+
|animation_original_url|animation_url|      asset_contract|background_color|          collection|             creator|decimals|description|       external_link|    id|  image_original_url|   image_preview_url| image_thumbnail_url|           image_url|is_presale|           last_sale|listing_date|            name|num_sales|               owner|           permalink|sell_orders|token_id|token_metadata|top_bid|              traits|transfer_fee|transfer_fee_payment_token|
+----------------------+-------------+--------------------

In [8]:
df = spark.read.json(json_lst)         

In [9]:
df1=df.select(
    df['asset_contract']['name'].alias('NFT'),\
    df['token_id'],\
    df['num_sales'],\
    df['owner']['user']['username'].alias('username'),\
    df['owner']['address'].alias('owner_address'),\
    to_timestamp(df['last_sale']['event_timestamp']).alias('txn_date'),\
    (df['last_sale']['total_price']/10**18).alias('payment_amt'),\
    df['last_sale']['payment_token']['symbol'].alias('payment_type'))

In [10]:
df1.show()

+---------+--------+---------+-------------+--------------------+-------------------+-----------+------------+
|      NFT|token_id|num_sales|     username|       owner_address|           txn_date|payment_amt|payment_type|
+---------+--------+---------+-------------+--------------------+-------------------+-----------+------------+
|Cool Cats|    1490|        1| dontbotherme|0x762b35b809ac426...|2021-10-05 01:44:29|      320.0|         ETH|
|Cool Cats|    3330|        1|   CoinUnited|0x4c4a5490deefefa...|2021-08-20 00:45:13|      110.0|         ETH|
|Cool Cats|    5635|        3|         null|0xe13756351f9cbc4...|2021-10-07 14:48:23|       99.0|         ETH|
|Cool Cats|    8624|        2|FriendlyDegen|0x2f5170deea82309...|2021-10-08 02:33:07|       83.0|        WETH|
|Cool Cats|    5280|        1|    SighVault|0xf5a9288eb6e86a3...|2021-08-07 17:52:48|       80.0|         ETH|
|Cool Cats|    2157|        3|  MR_CC_VAULT|0x9edf9b08406fa69...|2021-09-26 15:01:20|       77.0|         ETH|
|

In [11]:
df1.printSchema()

root
 |-- NFT: string (nullable = true)
 |-- token_id: string (nullable = true)
 |-- num_sales: long (nullable = true)
 |-- username: string (nullable = true)
 |-- owner_address: string (nullable = true)
 |-- txn_date: timestamp (nullable = true)
 |-- payment_amt: double (nullable = true)
 |-- payment_type: string (nullable = true)



In [10]:
reference_df=df1.select(
    df1['NFT'],\
    df1['owner_address'],\
    df1['txn_date']).cache()


CryptoCompare Historical USD Data

In [11]:
# converting to UNIX timestamp in order to make API calls for USD historical price
unix_df=reference_df.select(reference_df['NFT'],reference_df['txn_date']).withColumn('unix',unix_timestamp(reference_df['txn_date']))
unix_df.show()

+---------+-------------------+----------+
|      NFT|           txn_date|      unix|
+---------+-------------------+----------+
|Cool Cats|2021-10-05 01:44:29|1633423469|
|Cool Cats|2021-08-20 00:45:13|1629445513|
|Cool Cats|2021-10-07 14:48:23|1633643303|
|Cool Cats|2021-10-08 02:33:07|1633685587|
|Cool Cats|2021-08-07 17:52:48|1628383968|
|Cool Cats|2021-09-26 15:01:20|1632693680|
|Cool Cats|2021-08-28 17:17:38|1630196258|
|Cool Cats|2021-08-01 16:38:00|1627861080|
|Cool Cats|2021-10-06 18:26:08|1633569968|
|Cool Cats|2021-08-24 21:55:00|1629867300|
|Cool Cats|2021-09-26 15:01:20|1632693680|
|Cool Cats|2021-10-06 18:24:17|1633569857|
|Cool Cats|2021-09-26 15:14:46|1632694486|
|Cool Cats|2021-10-06 18:22:39|1633569759|
|Cool Cats|2021-10-06 18:32:38|1633570358|
|Cool Cats|2021-09-26 08:12:31|1632669151|
|Cool Cats|2022-01-10 15:38:58|1641857938|
|Cool Cats|2021-09-26 18:56:29|1632707789|
|Cool Cats|2021-10-05 04:44:22|1633434262|
|Cool Cats|2022-01-02 22:54:39|1641192879|
+---------+

In [13]:
def usd_convert(unix):
    cryptocompare_url=f'https://min-api.cryptocompare.com/data/pricehistorical?fsym=ETH&tsyms=USD&ts={unix}'
    cryptocompare_api_key='your-api-key'
    limit_exceeded=True
    while limit_exceeded==True:
        response=requests.get(cryptocompare_url,params={'api_key':cryptocompare_api_key})
        message=response.json()
        if message['ETH']:
            limit_exceeded=False
            return message['ETH']['USD']
        else:
            time.sleep(0.5)
            continue

dollar_udf=udf(lambda x : usd_convert(x))
dollar_df=unix_df.withColumn('usd_rate',dollar_udf(unix_df['unix'])).drop(unix_df['unix']).cache()

dollar_df.show()   


+---------+-------------------+--------+
|      NFT|           txn_date|usd_rate|
+---------+-------------------+--------+
|Cool Cats|2021-10-05 01:44:29| 3516.09|
|Cool Cats|2021-08-20 00:45:13|  3286.5|
|Cool Cats|2021-10-07 14:48:23| 3588.12|
|Cool Cats|2021-10-08 02:33:07| 3562.84|
|Cool Cats|2021-08-07 17:52:48| 3013.75|
|Cool Cats|2021-09-26 15:01:20| 3064.21|
|Cool Cats|2021-08-28 17:17:38|  3225.3|
|Cool Cats|2021-08-01 16:38:00| 2556.41|
|Cool Cats|2021-10-06 18:26:08| 3588.12|
|Cool Cats|2021-08-24 21:55:00| 3228.86|
|Cool Cats|2021-09-26 15:01:20| 3064.21|
|Cool Cats|2021-10-06 18:24:17| 3588.12|
|Cool Cats|2021-09-26 15:14:46| 3064.21|
|Cool Cats|2021-10-06 18:22:39| 3588.12|
|Cool Cats|2021-10-06 18:32:38| 3588.12|
|Cool Cats|2021-09-26 08:12:31| 3064.21|
|Cool Cats|2022-01-10 15:38:58| 3083.68|
|Cool Cats|2021-09-26 18:56:29| 2927.73|
|Cool Cats|2021-10-05 04:44:22| 3516.09|
|Cool Cats|2022-01-02 22:54:39| 3765.16|
+---------+-------------------+--------+
only showing top

Etherscan API Ethereum and Token Data

In [12]:
eth_addr_df=reference_df.select(reference_df['NFT'],reference_df['owner_address']).dropDuplicates()
eth_addr_df.show()

+---------+--------------------+
|      NFT|       owner_address|
+---------+--------------------+
|Cool Cats|0x8488f78d943cf6b...|
|Cool Cats|0x4005fbb366198c3...|
|Cool Cats|0xdb6d9af38ecadaf...|
|Cool Cats|0x40c839b831c9017...|
|Cool Cats|0x34978faf3a9f469...|
|Cool Cats|0xc96d32f17fa385d...|
|Cool Cats|0xcc93935c431cafc...|
|Cool Cats|0x32ae912d4abfde3...|
|Cool Cats|0xe7fa846ad3d12d8...|
|Cool Cats|0x7499c6c5d844fca...|
|Cool Cats|0x9edf9b08406fa69...|
|Cool Cats|0xad8357353ddf809...|
|Cool Cats|0x3adc1e9b6c09c1f...|
|Cool Cats|0x2d8d75d4ba64efd...|
|Cool Cats|0x9dbe56e65961146...|
|Cool Cats|0xffa914c83d851b9...|
|Cool Cats|0x2f5170deea82309...|
|Cool Cats|0x904778b44bd12a1...|
|Cool Cats|0x65de7da4eba5ed2...|
|Cool Cats|0xabf107de3e01c7c...|
+---------+--------------------+
only showing top 20 rows



In [15]:
def eth_balance(eth_address):
    etherscan_url='https://api.etherscan.io/api?'
    eth_api_key='your-api-key'
    param={'ETH_balance':{'module':'account','action':'balance','address':eth_address,'tag':'latest','apikey':eth_api_key}}
    
    limit_exceeded=True
    while limit_exceeded==True:
        response=requests.get(etherscan_url,params=param['ETH_balance'])
        message=response.json()
        if message['result']=='Max rate limit reached':
            time.sleep(0.5)
            continue
        else:
            limit_exceeded=False
            return message['result']

eth_udf=udf(lambda x : eth_balance(x))
eth_balance_df=eth_addr_df.withColumn('ETH',eth_udf(eth_addr_df['owner_address'])).cache()
eth_balance_df.show()

+-----------------+--------------------+--------------------+
|              NFT|       owner_address|                 ETH|
+-----------------+--------------------+--------------------+
|          Doodles|0xb0b22713b38adda...|   52784437803114463|
|      CryptoPunks|0x3b93cbfb99560ab...| 5022092324248575523|
|        Cool Cats|0x8488f78d943cf6b...| 6766267005198920729|
|        Cool Cats|0x4005fbb366198c3...|  361024938408471476|
|        Cool Cats|0xdb6d9af38ecadaf...|  468300529000000000|
|          Doodles|0xb5696e4057b9ba7...|  280763976745993296|
|      CryptoPunks|0x9c5083dd4838e12...|18546569427421342...|
|          Doodles|0xb4460bdf3fe5ba4...|10122542808237598484|
|        Cool Cats|0x40c839b831c9017...|45096293962530705030|
|BoredApeYachtClub|0x7eb413211a9de1c...| 1855774974803236227|
|        Cool Cats|0x34978faf3a9f469...| 8836151672031266571|
|      CryptoPunks|0x266892ed0d40ea5...|                   0|
|          Doodles|0xc35f3f92a9f27a1...| 4273770677697573350|
|      C

In [16]:

def token_balance(eth_address):
    etherscan_url='https://api.etherscan.io/api?'
    eth_api_key='your-api-key'
    token_dict={
    'Wrapped_eth':'0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
    'Tether':'0xdac17f958d2ee523a2206206994597c13d831ec7',
    'Usdc':'0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    'Dai':'0x6b175474e89094c44da98b954eedeac495271d0f',}
    new_dict={}
    for i in token_dict.keys():
        param={'token_balance':{'module':'account','action':'tokenbalance','contractaddress':token_dict[i],'address':eth_address,'tag':'latest','apikey':eth_api_key}}
        limit_exceeded=True
        while limit_exceeded==True:
            response=requests.get(etherscan_url,params=param['token_balance'])
            message=response.json()
            if message['result']=='Max rate limit reached':
                time.sleep(0.5)
                continue
            else:
                limit_exceeded=False
                new_dict[i]=message['result']
        
    return new_dict


token_udf=udf(lambda x : token_balance(x),MapType(StringType(),StringType()))
token_balance_df=eth_balance_df.withColumn('token_balance',token_udf(eth_balance_df['owner_address'])).cache()
token_balance_df.show()

+-----------------+--------------------+--------------------+--------------------+
|              NFT|       owner_address|                 ETH|       token_balance|
+-----------------+--------------------+--------------------+--------------------+
|          Doodles|0xb0b22713b38adda...|   52784437803114463|{Usdc -> 0, Dai -...|
|      CryptoPunks|0x3b93cbfb99560ab...| 5022092324248575523|{Usdc -> 55748400...|
|        Cool Cats|0x8488f78d943cf6b...| 6766267005198920729|{Usdc -> 98985258...|
|        Cool Cats|0x4005fbb366198c3...|  361024938408471476|{Usdc -> 0, Dai -...|
|        Cool Cats|0xdb6d9af38ecadaf...|  468300529000000000|{Usdc -> 0, Dai -...|
|          Doodles|0xb5696e4057b9ba7...|  280763976745993296|{Usdc -> 0, Dai -...|
|      CryptoPunks|0x9c5083dd4838e12...|18546569427421342...|{Usdc -> 14412232...|
|          Doodles|0xb4460bdf3fe5ba4...|10122542808237598484|{Usdc -> 0, Dai -...|
|        Cool Cats|0x40c839b831c9017...|45096293962530705030|{Usdc -> 30000000...|
|Bor

In [18]:
total_balance_df=token_balance_df.select(\
    token_balance_df['NFT'],\
    token_balance_df['owner_address'],\
    format_number((token_balance_df['ETH'].cast(FloatType())/10**18),2).alias('ETH'),\
    format_number(token_balance_df['token_balance']['Wrapped_eth'].cast(FloatType())/10**18,2).alias('Wrapped_ETH'),\
    format_number(token_balance_df['token_balance']['Dai'].cast(FloatType())/10**18,2).alias('DAI'),\
    format_number(token_balance_df['token_balance']['Usdc'].cast(FloatType())/10**6,2).alias('USDC'),\
    format_number(token_balance_df['token_balance']['Tether'].cast(FloatType())/10**6,2).alias('Tether')
    ).cache()


total_balance_df.show()


+-----------------+--------------------+------+-----------+-----+---------+---------+
|              NFT|       owner_address|   ETH|Wrapped_ETH|  DAI|     USDC|   Tether|
+-----------------+--------------------+------+-----------+-----+---------+---------+
|          Doodles|0xb0b22713b38adda...|  0.05|       0.00| 0.00|     0.00|     0.00|
|      CryptoPunks|0x3b93cbfb99560ab...|  5.02|       0.00| 0.00|55,748.40|     0.00|
|        Cool Cats|0x8488f78d943cf6b...|  6.77|       7.35| 0.00|98,985.25|     0.00|
|        Cool Cats|0x4005fbb366198c3...|  0.36|       0.00| 0.00|     0.00|     0.00|
|        Cool Cats|0xdb6d9af38ecadaf...|  0.47|       0.00| 0.00|     0.00|     0.00|
|          Doodles|0xb5696e4057b9ba7...|  0.28|       0.00| 0.00|     0.00|     0.00|
|      CryptoPunks|0x9c5083dd4838e12...|185.47|       0.00| 0.00|14,412.23|     0.00|
|          Doodles|0xb4460bdf3fe5ba4...| 10.12|       0.00| 0.00|     0.00|     0.00|
|        Cool Cats|0x40c839b831c9017...| 45.10|       